In [10]:
import pickle
import pandas as pd
from os.path import join
import os
import numpy as np
# Sk learn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [12]:
from scipy.stats import ttest_ind

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
%config InlineBackend.figure_format =   'svg'

CT_blue = '#4a79a5'
CT_gray = '#848688'
CT_pink = '#da6388'
CT_orig = '#3c71a3'

In [66]:
data_folder = '../data/preprocessed/'
X = pd.read_pickle(join(data_folder, "X.pk.zip"))
y = pd.read_pickle(join(data_folder, "Y.pk.zip"))

X_mat = X.values
y_vec = y.values.flatten()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y_vec, test_size=0.2, random_state=123)

In [67]:
X.shape

(70711, 189)

In [51]:
# X

## Difference in values

In [68]:
m0 = y_vec < 0.9
m1 = (y_vec > 0.9) * (y_vec < 1.1)
m2 = y_vec > 1.9

In [33]:
# X.columns[:10]

In [69]:
pvalt = 0.05
print('descr\t0-1\t0-2\t1-2')
for i in X.columns:
    par = i
    res0 = ttest_ind(X.loc[m0, [par]], X.loc[m1, [par]])[0][0]
    res1 = ttest_ind(X.loc[m0, [par]], X.loc[m2, [par]])[0][0]
    res2 = ttest_ind(X.loc[m2, [par]], X.loc[m1, [par]])[0][0]
    if (abs(res0) < pvalt) | (abs(res1) < pvalt) | (abs(res2) < pvalt) :
        print(f'{i}\t{res0:6.3f}\t{res1:6.3f}\t{res2:6.3f}')
# res1 = ttest_ind(X.loc[m1, [par]], y[m1])[0][0]

descr	0-1	0-2	1-2
Weta3.unity	-1.108	 0.019	-1.738
FMF	-1.704	 0.008	-2.794
khs.sCH3	 1.772	 2.326	 0.042
khs.tCH	 0.031	 2.538	-2.897
khs.ssNH2	-0.351	-0.455	 0.006
khs.aaN	-0.726	 0.045	-1.174
khs.ssO	 0.011	 1.144	-1.323
khs.aaS	 0.119	 0.136	 0.024
khs.ddssS	-0.027	-0.579	 0.640
MOMI-XY	-0.042	 0.832	-1.038
nRings5	 0.007	-1.062	 1.257


In [54]:
res0, res1, res2

(0.014242191899012591, 0.02855601805445669, -0.01202994957471932)

## Classification

In [32]:
# X

In [70]:
scaler = StandardScaler()

In [71]:
scaler = scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)

In [72]:
features = X.columns.values
features.shape

(189,)

### Random Forest

In [73]:
cl =  RandomForestClassifier(class_weight='balanced')
pred = cl.fit(X_train_norm, y_train)

In [78]:
features[cl.feature_importances_ > 0.015]

array(['ALogP', 'ALogp2', 'Wlambda1.unity', 'Wnu1.unity', 'Weta2.unity',
       'Weta3.unity', 'MOMI-XY', 'geomShape', 'XLogP'], dtype=object)

### Decision Tree

In [79]:
cl = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
pred = cl.fit(X_train_norm, y_train)

In [80]:
features[cl.feature_importances_ > 0.05]

array(['ALogp2', 'Wlambda1.unity', 'nAromBond', 'XLogP'], dtype=object)